In [27]:
import os 

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit

In [28]:
pandas = pd.read_table("lpmc02.dat")
database = db.Database("LPMC",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

# Choice
chosenAlternative = travel_mode

In [29]:
#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1 = Beta('Constant1',0,None,None,1)
Constant2 = Beta('Constant2',0,None,None,0)
Constant3 = Beta('Constant3',0,None,None,0)
Constant4 = Beta('Constant4',0,None,None,0)
Cost = Beta('Cost',0,None,None,0)
Total_TT1 = Beta('Total_TT1',0,None,None,0)
Total_TT2 = Beta('Total_TT2',0,None,None,0)
Total_TT3 = Beta('Total_TT3',0,None,None,0)
Total_TT4 = Beta('Total_TT4',0,None,None,0)
CarOwn = Beta('CarOwn',0,None,None,0)


# socio-economic factors (interacting with CarOwn)
CarOwn_Female = Beta('Dist_Female', 0, None, None, 0)
CarOwn_Age = Beta('Dist_Age', 0, None, None, 0)

In [30]:
# Define here arithmetic expressions for name that are not directly
# available from the data

dur_public = DefineVariable('dur_public', dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int ,database)
cost_public = DefineVariable('cost_public', cost_transit ,database)
cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)

##### Model with Gender as the socio-econ interaction

In [31]:
# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving

Opt1 = Constant1 + Total_TT1 * dur_walking + CarOwn * distance + CarOwn_Female * distance * female
Opt2 = Constant2 + Total_TT2 * dur_cycling + CarOwn * distance + CarOwn_Female * distance * female
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * dur_public + CarOwn * distance +\
                    CarOwn_Female * distance * female
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * dur_driving + CarOwn * car_ownership +\
                    CarOwn * distance + CarOwn_Female * distance * female
    
V = {1: Opt1,2: Opt2,3: Opt3, 4: Opt4}
av = {1: 1,2: 1,3: 1,4: 1}

In [ ]:
output_dir = "./model2-output"
filepath = os.path.join(output_dir, "logit_lpmc_model2_CarOwn+Gender")

if not os.path.exists(output_dir):
    os.mkdir(output_dir)
if os.path.exists(filepath):
    os.remove(f"{filepath}.html")
    
# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = filepath
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

pandasResults

##### Model with Age as the socio-econ interaction

In [ ]:
# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving

Opt1 = Constant1 + Total_TT1 * dur_walking + CarOwn * distance + CarOwn_Age * distance * age
Opt2 = Constant2 + Total_TT2 * dur_cycling + CarOwn * distance + CarOwn_Age * distance * age
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * dur_public + CarOwn * distance +\
                    CarOwn_Age * distance * age
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * dur_driving + CarOwn * car_ownership +\
                    CarOwn * distance + CarOwn_Age * distance * age
    
V = {1: Opt1,2: Opt2,3: Opt3, 4: Opt4}
av = {1: 1,2: 1,3: 1,4: 1}

In [ ]:
output_dir = "./model2-output"
filepath = os.path.join(output_dir, "logit_lpmc_model2_CarOwn+Age")

if not os.path.exists(output_dir):
    os.mkdir(output_dir)
if os.path.exists(filepath):
    os.remove(f"{filepath}.html")
    
# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = filepath
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

pandasResults